In [2]:
import pandas as pd
import numpy as np 
from torch.utils.data import DataLoader, Dataset
import random as random
from torch import cat, nn, mul, transpose


In [9]:
ratings_df = pd.read_csv('/Users/60113/dataset/ml-25m/ratings.csv')
movies_df = pd.read_csv('/Users/60113/dataset/ml-25m/movies.csv')


In [11]:
temp = ratings_df['userId'].dropna().unique()
temp

array([     1,      2,      3, ..., 162539, 162540, 162541], dtype=int64)

In [4]:
valid_users[['userId', 'counts']] = valid_users[['userId', 'counts']].apply(pd.to_numeric)
valid_users = valid_users.loc[valid_users['counts'] > 20]
movies_ratings_df[['movieId','userId','rating']] = movies_ratings_df[['movieId','userId','rating']].apply(pd.to_numeric)
valid_users.shape

(9663, 2)

In [5]:
# # Remove timestamp column
# del movies_ratings_df['timestamp']
# full_data_Valid_Users = full_data[full_data.User_id.isin(Valid_Users.User_id)]
# movies_ratings_df = pd.merge(movies_ratings_df, valid_users, on='userId')
# movies_ratings_df.set_index('userId',inplace=True)

movies_ratings_df = movies_ratings_df[['userId', 'movieId', 'rating']]
movies_ratings_df = movies_ratings_df[movies_ratings_df['userId'].isin(valid_users['userId'])]
# movies_ratings_df[movies_ratings_df > 1] = 1

# movies_ratings_df['userId'] = movies_ratings_df['userId'].astype(int)
# movies_ratings_df.shape
# # pivot_table = movies_ratings_df.pivot_table(index='title', columns='userId', values='rating')
# # pivot_table.head()

print(movies_ratings_df.shape)

In [6]:
movies_ratings_df.loc[movies_ratings_df['rating'] > 0, 'rating'] = 1.0
movies_ratings_df.head()

,userId,movieId,rating
1,3,1,1.0
6,12,1,1.0
35,120,1,1.0
47,166,1,1.0
49,171,1,1.0


In [7]:
# rename movieId 
movies_ratings_df = movies_ratings_df.rename(columns={'movieId':'itemId'})
user_pool, item_pool = set(movies_ratings_df['userId'].unique()), set(movies_ratings_df['itemId'].unique())

user_item_interaction = movies_ratings_df.groupby('userId')['itemId'].apply(set).reset_index().rename(
            columns = {'itemId':'interacted_items'})

# user_item_interaction['unobserved_items'] = user_item_interaction['interacted_items'].apply(lambda x : item_pool - x)
# user_item_interaction['unobserved_samples'] = user_item_interaction['unobserved_items'].apply(lambda x: random.sample(x, 99))

user_item_interaction


,userId,interacted_items
0,3,"{1, 122882, 122886, 102407, 69640, 6155, 6156,..."
1,12,"{1, 2, 3, 2052, 4100, 2054, 7, 4103, 4104, 10,..."
2,72,"{2, 2053, 6, 7, 2054, 4102, 11, 2059, 4109, 16..."
3,80,"{2, 6, 2054, 10, 2058, 16, 21, 22, 23, 24, 31,..."
4,120,"{2048, 1, 2049, 2050, 7, 2058, 11, 2059, 2064,..."
...,...,...
9658,162484,"{2048, 1, 2, 4, 5, 2053, 2054, 10, 11, 12, 19,..."
9659,162495,"{1, 2, 3, 5, 40966, 10, 13, 15, 19, 57368, 32,..."
9660,162508,"{1, 6, 8198, 14, 8207, 16, 17, 32792, 25, 5736..."
9661,162516,"{1, 2, 3, 4, 8196, 6, 7, 122882, 122886, 10, 1..."


In [8]:
user_item_interaction['unobserved_items'] = user_item_interaction['interacted_items'].apply(lambda x : item_pool - x)


In [9]:
user_item_interaction

,userId,interacted_items,unobserved_items
0,3,"{1, 122882, 122886, 102407, 69640, 6155, 6156,...","{131072, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1..."
1,12,"{1, 2, 3, 2052, 4100, 2054, 7, 4103, 4104, 10,...","{131072, 4, 5, 6, 8, 9, 11, 12, 13, 14, 15, 17..."
2,72,"{2, 2053, 6, 7, 2054, 4102, 11, 2059, 4109, 16...","{131072, 1, 3, 4, 5, 8, 9, 10, 12, 13, 14, 15,..."
3,80,"{2, 6, 2054, 10, 2058, 16, 21, 22, 23, 24, 31,...","{131072, 1, 3, 4, 5, 7, 8, 9, 11, 12, 13, 14, ..."
4,120,"{2048, 1, 2049, 2050, 7, 2058, 11, 2059, 2064,...","{131072, 2, 3, 4, 5, 6, 8, 9, 10, 12, 13, 14, ..."
...,...,...,...
9658,162484,"{2048, 1, 2, 4, 5, 2053, 2054, 10, 11, 12, 19,...","{131072, 3, 6, 7, 8, 9, 13, 14, 15, 16, 17, 18..."
9659,162495,"{1, 2, 3, 5, 40966, 10, 13, 15, 19, 57368, 32,...","{131072, 4, 6, 7, 8, 9, 11, 12, 14, 16, 17, 18..."
9660,162508,"{1, 6, 8198, 14, 8207, 16, 17, 32792, 25, 5736...","{131072, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, ..."
9661,162516,"{1, 2, 3, 4, 8196, 6, 7, 122882, 122886, 10, 1...","{131072, 5, 8, 9, 12, 26, 27, 28, 30, 33, 35, ..."


In [11]:
user_item_interaction['unobserved_samples'] = user_item_interaction['unobserved_items'].apply(lambda x: random.sample(list(x), 99))
user_item_interaction

,userId,interacted_items,unobserved_items,unobserved_samples
0,3,"{1, 122882, 122886, 102407, 69640, 6155, 6156,...","{131072, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1...","[94959, 147346, 2666, 203447, 198117, 194847, ..."
1,12,"{1, 2, 3, 2052, 4100, 2054, 7, 4103, 4104, 10,...","{131072, 4, 5, 6, 8, 9, 11, 12, 13, 14, 15, 17...","[26389, 166749, 171775, 94748, 72405, 172135, ..."
2,72,"{2, 2053, 6, 7, 2054, 4102, 11, 2059, 4109, 16...","{131072, 1, 3, 4, 5, 8, 9, 10, 12, 13, 14, 15,...","[26049, 199360, 190759, 151404, 118522, 87262,..."
3,80,"{2, 6, 2054, 10, 2058, 16, 21, 22, 23, 24, 31,...","{131072, 1, 3, 4, 5, 7, 8, 9, 11, 12, 13, 14, ...","[126086, 199746, 168622, 161838, 5574, 7355, 1..."
4,120,"{2048, 1, 2049, 2050, 7, 2058, 11, 2059, 2064,...","{131072, 2, 3, 4, 5, 6, 8, 9, 10, 12, 13, 14, ...","[195781, 116343, 168164, 32686, 154854, 178679..."
...,...,...,...,...
9658,162484,"{2048, 1, 2, 4, 5, 2053, 2054, 10, 11, 12, 19,...","{131072, 3, 6, 7, 8, 9, 13, 14, 15, 16, 17, 18...","[8482, 173277, 25957, 4432, 164055, 94473, 149..."
9659,162495,"{1, 2, 3, 5, 40966, 10, 13, 15, 19, 57368, 32,...","{131072, 4, 6, 7, 8, 9, 11, 12, 14, 16, 17, 18...","[159469, 200188, 77667, 26082, 55080, 161356, ..."
9660,162508,"{1, 6, 8198, 14, 8207, 16, 17, 32792, 25, 5736...","{131072, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, ...","[133379, 135216, 141870, 2579, 173893, 152163,..."
9661,162516,"{1, 2, 3, 4, 8196, 6, 7, 122882, 122886, 10, 1...","{131072, 5, 8, 9, 12, 26, 27, 28, 30, 33, 35, ...","[6846, 168276, 124275, 2036, 176749, 175525, 1..."
